# CBOR vs JSON

Anton Antonov   
September 2024

-------

## Setup

In [14]:
use LLM::RetrievalAugmentedGeneration;
use LLM::RetrievalAugmentedGeneration::VectorDatabase;

use Data::Importers;
use Data::Reshapers;
use Data::Summarizers;

use XDG::BaseDirectory :terms;

use JSON::Fast;
use CBOR::Simple;

use NativeCall;

------

## Vector databases

Skim vector databases from the default directory and show summaries:

In [3]:
#% html
my @field-names = <id name item-count dimension version llm-service llm-embedding-model created>;
vector-database-objects(f=>'hash', :flat)
==> { $_.map({ $_<created> = $_<file>.IO.created.DateTime.Str.subst('T',' ').substr(^19); $_}).sort(*<created>).reverse }()
==> to-html(:@field-names)

id name item-count dimension version llm-service llm-embedding-model created f00209fa-0f35-41bf-8597-744dae9aab5d No833 283 4096 0 llama text-embedding-3-small 2024-09-19 01:31:12 a6b12f80-0ed7-4125-9e2b-9f77c11a16f8 No833 441 4096 0 llama text-embedding-3-small 2024-09-19 01:26:58 32bd53cb-7171-4d26-9a4f-11de377d3af1 words 20 4096 0 llama text-embedding-3-small 2024-09-19 01:14:27 5097c865-7fad-43d9-b7ca-456220d754b7 words 20 768 0 gemini embedding-001 2024-09-18 14:08:44 045f467c-193f-4df6-bec3-790d6c83ca64 No833 441 768 0 gemini embedding-001 2024-09-16 22:21:25 d2effebc-2cef-4b2b-84ca-5dcfa3c1864b No747 283 1536 0 chatgpt text-embedding-3-small 2024-09-12 13:32:30 5cb40fbb-9f69-48ca-9fc1-03ec8059ed99 No747 283 768 0 gemini embedding-001 2024-09-12 13:32:23 44f19858-730e-4b96-86b7-81e701f9df8f No747 283 1536 0 chatgpt text-embedding-3-small 2024-09-12 13:32:18 266b20ca-d917-4ac0-9b0a-7c420625666c No833 441 1536 0 chatgpt text-embedding-3-small 2024-09-12 13:32:11

-----

## Import from JSON format

Get specific databases:

In [4]:
my @vdbs = vector-database-objects(f=>'hash', :flat).grep({ $_<id> ∈ <266b20ca-d917-4ac0-9b0a-7c420625666c d2effebc-2cef-4b2b-84ca-5dcfa3c1864b> }).map({ create-vector-database(file => $_<file>) })

[VectorDatabase(:id("266b20ca-d917-4ac0-9b0a-7c420625666c"), :name("No833"), :elements(441), :sources(442), :precision(num64)) VectorDatabase(:id("d2effebc-2cef-4b2b-84ca-5dcfa3c1864b"), :name("No747"), :elements(283), :sources(284), :precision(num64))]

-----

## Export to CBOR format

In [6]:
@vdbs.head.export(format => 'cbor')

VectorDatabase(:id("266b20ca-d917-4ac0-9b0a-7c420625666c"), :name("No833"), :elements(441), :sources(442), :precision(num64))

In [9]:
my $vdbObj = LLM::RetrievalAugmentedGeneration::VectorDatabase.new();
$vdbObj.import('/Users/antonov/.local/share/raku/LLM/SemanticSearchIndex/SemSe-266b20ca-d917-4ac0-9b0a-7c420625666c.cbor')

VectorDatabase(:id("266b20ca-d917-4ac0-9b0a-7c420625666c"), :name("No833"), :elements(441), :sources(442), :precision(num64))

In [10]:
my $vdbObj2 = LLM::RetrievalAugmentedGeneration::VectorDatabase.new(num-type => 'num32');
$vdbObj2.import('/Users/antonov/.local/share/raku/LLM/SemanticSearchIndex/SemSe-266b20ca-d917-4ac0-9b0a-7c420625666c.json')

VectorDatabase(:id("266b20ca-d917-4ac0-9b0a-7c420625666c"), :name("No833"), :elements(441), :sources(442), :precision(num64))

In [12]:
$vdbObj2.vectors

{001.0 => NativeCall::Types::CArray[num64].new, 002.0 => NativeCall::Types::CArray[num64].new, 003.0 => NativeCall::Types::CArray[num64].new, 004.0 => NativeCall::Types::CArray[num64].new, 005.0 => NativeCall::Types::CArray[num64].new, 006.0 => NativeCall::Types::CArray[num64].new, 007.0 => NativeCall::Types::CArray[num64].new, 008.0 => NativeCall::Types::CArray[num64].new, 009.0 => NativeCall::Types::CArray[num64].new, 010.0 => NativeCall::Types::CArray[num64].new, 011.0 => NativeCall::Types::CArray[num64].new, 012.0 => NativeCall::Types::CArray[num64].new, 013.0 => NativeCall::Types::CArray[num64].new, 014.0 => NativeCall::Types::CArray[num64].new, 015.0 => NativeCall::Types::CArray[num64].new, 016.0 => NativeCall::Types::CArray[num64].new, 017.0 => NativeCall::Types::CArray[num64].new, 018.0 => NativeCall::Types::CArray[num64].new, 019.0 => NativeCall::Types::CArray[num64].new, 020.0 => NativeCall::Types::CArray[num64].new, 021.0 => NativeCall::Types::CArray[num64].new, 022.0 => Nat

In [19]:
my %vectors = $vdbObj2.vectors;
%vectors .= map({ $_.key => CArray[num32].new($_.value) });

{001.0 => NativeCall::Types::CArray[num32].new, 002.0 => NativeCall::Types::CArray[num32].new, 003.0 => NativeCall::Types::CArray[num32].new, 004.0 => NativeCall::Types::CArray[num32].new, 005.0 => NativeCall::Types::CArray[num32].new, 006.0 => NativeCall::Types::CArray[num32].new, 007.0 => NativeCall::Types::CArray[num32].new, 008.0 => NativeCall::Types::CArray[num32].new, 009.0 => NativeCall::Types::CArray[num32].new, 010.0 => NativeCall::Types::CArray[num32].new, 011.0 => NativeCall::Types::CArray[num32].new, 012.0 => NativeCall::Types::CArray[num32].new, 013.0 => NativeCall::Types::CArray[num32].new, 014.0 => NativeCall::Types::CArray[num32].new, 015.0 => NativeCall::Types::CArray[num32].new, 016.0 => NativeCall::Types::CArray[num32].new, 017.0 => NativeCall::Types::CArray[num32].new, 018.0 => NativeCall::Types::CArray[num32].new, 019.0 => NativeCall::Types::CArray[num32].new, 020.0 => NativeCall::Types::CArray[num32].new, 021.0 => NativeCall::Types::CArray[num32].new, 022.0 => Nat

In [20]:
my $vdbObj3 = LLM::RetrievalAugmentedGeneration::VectorDatabase.new(num-type => 'num32', :%vectors, items => $vdbObj2.items);

VectorDatabase(:id("77da9a57-b53f-4b93-9935-ffa5476b8d7b"), :name(""), :elements(441), :sources(441), :precision(num32))

In [22]:
$vdbObj3.export(format => 'cbor')

VectorDatabase(:id("77da9a57-b53f-4b93-9935-ffa5476b8d7b"), :name(""), :elements(441), :sources(441), :precision(num32))

In [26]:
my $dirname = LLM::RetrievalAugmentedGeneration::default-location();
my $vdbObj4 = create-vector-database(file => $dirname ~ "/SemSe-77da9a57-b53f-4b93-9935-ffa5476b8d7b.cbor")

Cannot ingest as a ⎡cbor⎦ file: ⎡/Users/antonov/.local/share/raku/LLM/SemanticSearchIndex/SemSe-77da9a57-b53f-4b93-9935-ffa5476b8d7b.cbor⎦.

Unassigned simple value 4
  in sub cbor-decode at /Users/antonov/.rakubrew/versions/moar-2024.08/share/perl6/site/sources/3CE80F14BC4AC546C7CFF3C3860B184EDC30A0C9 (CBOR::Simple) line 587
  in sub cbor-decode at /Users/antonov/.rakubrew/versions/moar-2024.08/share/perl6/site/sources/3CE80F14BC4AC546C7CFF3C3860B184EDC30A0C9 (CBOR::Simple) line 575
  in code  at /Users/antonov/.rakubrew/versions/moar-2024.08/share/perl6/site/sources/6DF4B6B100FF42ED86B02D68846150B0C1074965 (LLM::RetrievalAugmentedGeneration::VectorDatabase) line 464
  in method import at /Users/antonov/.rakubrew/versions/moar-2024.08/share/perl6/site/sources/6DF4B6B100FF42ED86B02D68846150B0C1074965 (LLM::RetrievalAugmentedGeneration::VectorDatabase) line 459
  in sub create-vector-database at /Users/antonov/.rakubrew/versions/moar-2024.08/share/perl6/site/sources/0CA064A7A6395EA0BA47C67138FE2EF5CE0CBEA6 (LLM::RetrievalAugmentedGeneration) line 73
  in block <unit> at <unknown file> line 1
  in method eval at /Users/antono